In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.2/220.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.utils.math import cosine_similarity
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
import pandas as pd
import numpy as np
import os
import pickle

In [ ]:
# OpenAI API Keyを環境変数に保存
from getpass import getpass
os.environ['OPENAI_API_KEY'] = getpass('OpenAI API Key:')

OpenAI API Key:··········


In [5]:
# ディレクトリの移動
os.chdir('/content/drive/MyDrive/aozorabunko')

In [ ]:
# 埋め込みモデルのインスタンスを作成
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
# チャットモデルのインスタンスを作成
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

## テキストの前処理

In [ ]:
import re
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# remove unneccesary lines and characters from .txt files
# ref: https://www.osaka-kyoiku.ac.jp/~kokugo/nonami/awk/rubycut_awk.html
files = [s for s in os.listdir('./files') if re.search('\.txt$', s) is not None]
docs = []
for file in files:
    with open('./files/' + file, 'r', encoding='SHIFT_JIS') as f:
        text = f.read()
    text = re.sub('\n-{30,}\n(.|\n)*?\n-{30,}\n', '', text)
    text = re.sub('［＃[^］]*?］', '', text) # ［＃…］を除く
    text = re.sub('\n(|翻訳の)底本：(.|\n)*$', '', text)
    text = re.sub('《[^》]+》', '', text) # ルビを除く
    text = re.sub('｜', '', text) # ルビの境界記号を除く
    text = re.sub('\n+$', '', text) # 最後の改行を除く
    # タイトル・作者と本文に分けDocumentsに保存
    title_author, text = text.split('\n\n', maxsplit=1)
    doc = Document(page_content=text, metadata={'title_author':title_author})
    docs.append(doc)


In [ ]:
# 本文の文字数とトークン数
import tiktoken
for doc in docs:
    title = doc.metadata['title_author'].split('\n')[0]
    nchar = len(doc.page_content)
    ntoken = len(tiktoken.encoding_for_model('text-embedding-ada-002').encode(doc.page_content))
    print('\t'.join([title, str(nchar), str(ntoken)]))


赤ずきんちゃん	3770	3744
シンデレラ	8448	8227
マッチ売りの少女	3487	3427
金太郎	3942	4059
瘤とり	4941	4964
猿かに合戦	3442	3452
浦島太郎	5707	5746
花咲かじじい	3132	3061
一寸法師	5692	5802
桃太郎	5605	5735


In [ ]:
tiktoken.encoding_for_model('text-embedding-ada-002').encode('桃太郎')

## map-reduce法で要約


https://python.langchain.com/docs/use_cases/summarization#option-2-map-reduce

In [ ]:
# gpt-3.5-turboでは入出力合わせてトークン数上限が4096のため，3000トークン以内に分割する．
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=3000,
    chunk_overlap=0
    )

In [ ]:
# 英語で返ってくる（日本語で返すこともあるかも）．
chain = load_summarize_chain(llm, chain_type="map_reduce")
split_docs = text_splitter.split_documents(docs[:1])
chain.run(split_docs) # トークン数4097以上だとエラー

'Little Red Riding Hood is tricked by a wolf who eats her grandmother and disguises himself as her. A hunter rescues them and they kill the wolf. Little Red Riding Hood learns her lesson and vows to stay on the path in the forest.'

In [ ]:
# 日本語で要約するために必要なモジュールをインポート
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
#from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain

In [ ]:
# Map chain
map_template = """次のテキストは物語の一部です．この部分のあらすじを400～500字に要約してください.
{doc}""" # 指定した字数での要約は難しいらしい．120字程度になることもある．
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [ ]:
# Reduce chain
reduce_template = """次のテキストは物語のあらすじです．これをさらに簡潔に400～500字で一つの段落にまとめてください．
{doc_summaries}""" # ここでトークン数上限を超えたらエラー？
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [ ]:
# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries" # document_variable_name="docs"だとエラーになった．間違いと思われるので変更
    )

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=2000
    )
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [ ]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="doc",
    # Return the results of the map steps in the output
    return_intermediate_steps=True,
    )


In [ ]:
# 要約の実行
import time
results = []
summary_docs = []
for doc in docs:
    start = time.time()
    print(doc.metadata['title_author'].split('\n')[0])
    split_docs = text_splitter.split_documents([doc])
    result = map_reduce_chain({"input_documents": split_docs}, return_only_outputs=True)
    results.append(result)
    doc = Document(page_content=result['output_text'],
                   metadata={'title_author': '[要約]' + doc.metadata['title_author']})
    summary_docs.append(doc)
    print(round((time.time() - start) / 60, 1))

赤ずきんちゃん
3.0
シンデレラ
8.1
マッチ売りの少女
1.5
金太郎
1.9
瘤とり
3.0
猿かに合戦
3.6
浦島太郎
4.5
花咲かじじい
2.9
一寸法師
4.8
桃太郎
2.7


In [ ]:
# 結果を保存
with open('./mapreduce_results.pkl', 'wb') as f:
    pickle.dump(results, f)
with open('./mapreduce_summary_docs.pkl', 'wb') as f:
    pickle.dump(summary_docs, f)

In [ ]:
# 要約結果を確認
summary_docs # 「瘤取り」の要約はハルシネーション

[Document(page_content='赤ずきんちゃんはおばあさんに可愛がられ、ずきんを作ってもらいました。ある日、お母さんがお菓子とぶどう酒をおばあさんに届けるよう頼みます。森の中で赤ずきんちゃんはおおかみに出会い、おばあさんの家までの道を教えますが、おおかみは赤ずきんちゃんを食べようとします。赤ずきんちゃんはお花を摘んでいる最中、おばあさんの家に戻りますが、おばあさんが変わった様子で横になっていました。おおかみが現れて赤ずきんちゃんを食べ、寝床に戻ります。かりうどが通りかかり、おおかみを見つけて鉄砲を向けますが、おばあさんがまだおなかの中にいるかもしれないと思い、おなかを切ります。赤ずきんちゃんとおばあさんは生きていて、おおかみを倒します。三人は喜び、かりうどはおおかみの毛皮を持ち帰ります。おばあさんはお菓子とぶどう酒を食べて元気を取り戻し、赤ずきんちゃんはもう二度と森に入らないことを決めます。', metadata={'title_author': '[要約]赤ずきんちゃん\nROTKAPPCHEN\nグリム兄弟\u3000Bruder Grimm\n楠山正雄訳'}),
 Document(page_content='昔々、高飛車な妻と結婚した男性の娘、シンデレラは姉たちにいじめられながらも美しい部屋とベッドで暮らしていた。ある日、王子のダンスパーティが開かれ、姉たちはドレスを選び始める。シンデレラは行きたくないと言うが、乳母の魔法でカボチャを馬車に変えられ、パーティに送り出される。シンデレラはハツカネズミを解放し、馬に変身させ、ドブネズミを運転手に、トカゲを従者に変える。パーティで王子様と出会い、時間を忘れてしまい、急いで逃げ出すが、ガラスの靴を残してしまう。王子様は靴を拾い、シンデレラを探し始める。最終的にシンデレラが靴を履くとぴったりとはまり、姉たちは驚き、シンデレラが本当の美しい姫だったことに気づく。姉たちは謝罪し、シンデレラは王子様と結婚し、姉たちも同じ日にお城の上流階級と結婚した。', metadata={'title_author': '[要約]シンデレラ\nCINDERELLA, OR THE LITTLE GLASS SLIPPER\n―ガラスのくつのものがたり―\nアンドルー・ラング再話\u3000Andrew Lang\n大久保ゆ

In [ ]:
# 要約の長さを確認
[len(doc.page_content) for doc in summary_docs] # プロンプトの指示より長くなっているものもある

[385, 344, 256, 358, 176, 645, 342, 726, 315, 427]

In [ ]:
# intermediate_stepsを見てみる
results = pd.read_pickle('./mapreduce_results.pkl')
results[6]

## refine法で要約

https://python.langchain.com/docs/use_cases/summarization#option-3-refine

In [ ]:
# gpt-3.5-turboでは入出力合わせてトークン数上限が4096のため，3000文字以内に分割する．
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=3000,
    chunk_overlap=50
    )

In [ ]:
# 日本語で400-500字に要約するようプロンプトを修正．
prompt_template = """次のテキストは物語の一部です．この部分のあらすじを400字から500字で要約してください.
{text}"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = ( # 日本語にするのが難しいのでプロンプトは英語のまま日本語出力するよう変更．
    "Your job is to produce a final summary.\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary into 400 - 500 characters in Japanese. "
    "If the context isn't useful, return the original summary."
    )
refine_prompt = PromptTemplate.from_template(refine_template)
refine_chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
    )



In [ ]:
# 要約を実行
results = []
summary_docs = []
for doc in docs:
    start = time.time()
    print(doc.metadata['title_author'].split('\n')[0])
    split_docs = text_splitter.split_documents([doc])
    result = refine_chain({"input_documents": split_docs}, return_only_outputs=True)
    results.append(result)
    doc = Document(page_content=result['output_text'],
                   metadata={'title_author': '[要約]' + doc.metadata['title_author']})
    summary_docs.append(doc)
    print(round((time.time() - start) / 60, 1))


In [ ]:
# 結果を保存
with open('refine_results.pkl', 'wb') as f:
    pickle.dump(results, f)
with open('refine_summary_docs.pkl', 'wb') as f:
    pickle.dump(summary_docs, f)

In [ ]:
# 要約結果を確認
summary_docs # 「シンデレラ」「猿かに合戦」「一寸法師」でチャンクごと情報が欠落，「瘤取り」の要約はハルシネーション

[Document(page_content='あるところに、かわいい赤ずきんちゃんという女の子がいました。彼女はおばあさんにとても可愛がられていました。ある日、おばあさんは赤ずきんちゃんに赤いずきんを作ってあげました。それ以来、彼女は赤ずきんちゃんと呼ばれるようになりました。赤ずきんちゃんはお母さんの頼みでお菓子とぶどう酒をおばあさんのところに届ける途中、おおかみに出会いました。おおかみは赤ずきんちゃんを食べようと考え、おばあさんの家に先回りしておばあさんを食べてしまいました。赤ずきんちゃんがおばあさんの家に着くと、おおかみがおばあさんのふりをして待っていました。その後、かりうどが通りかかり、おおかみを見つけて助け出しました。おばあさんも生きていて、三人は喜びました。', metadata={'title_author': '[要約]赤ずきんちゃん\nROTKAPPCHEN\nグリム兄弟\u3000Bruder Grimm\n楠山正雄訳'}),
 Document(page_content='シンデレラはパーティから帰宅し、元のぼろい服に戻っていたが、ガラスの靴だけが残っていた。姉たちはシンデレラの話を聞き、王子様が靴を見つけたことを伝えた。王子様は靴を持ち上げた。王子様は靴を試すために様々な女性に履かせたが、誰もぴったりとはまらなかった。靴はシンデレラの家にもやってきた。姉たちは靴を履こうとしたが、うまくいかなかった。シンデレラは靴を履いてみせ、ぴったりとはまった。姉たちは驚き、シンデレラを許しを請い、シンデレラは王子様のもとへ案内された。王子様はシンデレラが一番美しいと思った。数日後、シンデレラと王子様は結婚式を挙げた。姉たちもお城で暮らすようになり、結婚式の日に姉たちも立派な人と結婚した。', metadata={'title_author': '[要約]シンデレラ\nCINDERELLA, OR THE LITTLE GLASS SLIPPER\n―ガラスのくつのものがたり―\nアンドルー・ラング再話\u3000Andrew Lang\n大久保ゆう訳'}),
 Document(page_content='寒いおおみそかの晩、年のいかない少女はみすぼらしい格好で街を歩いていた。彼女は大きなスリッパをはいていたが、馬車にぶつかって失くしてしまった。そのため、

## 要約前の元のテキストをチャンクに分割

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500,
    chunk_overlap=0)
chunk_docs = text_splitter.split_documents(docs)
with open ('./chunk_docs.pkl', 'wb') as f:
    pickle.dump(chunk_docs, f)

## 埋め込みベクトルをndarrayで保存
（Chromaでトークン数上限以上の文章をベクトル化する方法がわからないので）




In [7]:
# map-reduce法による要約と元の本文のチャンクのDocumentを結合
with open('./mapreduce_summary_docs.pkl', 'rb') as f:
    summary_docs = pickle.load(f)
with open ('./chunk_docs.pkl', 'rb') as f:
    chunk_docs = pickle.load(f)
summary_chunk_docs = summary_docs + chunk_docs
with open('./summary_chunk_docs.pkl', 'wb') as f:
    pickle.dump(summary_chunk_docs, f)

len(summary_chunk_docs)

186

In [ ]:
# チャンクをベクトルに埋め込み，結合してアレイに
# [embeddings.embed_query(doc.page_content) for doc in summary_chunk_docs]だとRateLimitErrorになるのでsleepさせる
embeddings_list = []
time.sleep(420)
for i, doc in enumerate(summary_chunk_docs):
    print(i)
    embeddings_list.append(embeddings.embed_query(doc.page_content))
    time.sleep(420)

embeddings_array = np.array(embeddings_list)

# 結合したアレイを保存
with open('./summary_chunk_embeddings_array.pkl', 'wb') as f:
    pickle.dump(embeddings_array, f)
embeddings_array.shape

177
178
179
180
181
182
183
184
185


(186, 1536)